# Machine Learning Challenge

Below are 2 data challenges that test for your ability to:
- Wrangle/clean data to make it usable by a model
- Figure out how to set up X's and y's for a use case, given a dataset
- Write code to robustly and reproducibly preprocess data
- Pick/design the right model, and tune hyperparameters to get the best performance

You can use any programming language, model, and package to solve these problems. Let us know of any assumptions you make in your process.

#### Deliverables:
- A link to a github repository that contains:
    - Clearly commented code that was written to solve these problems
    - Your trained models stored in a file (`.pkl`, `.h5`, `.tar` - whatever is appropriate). The models must have `predict(X)` functions. 
    - A readme file that contains:
        - Instructions to easily access/load the above
        - A writeup explaining any significant design decisions and your reasons for making them. 
        - If needed, a brief writeup explaining anything you are particularly proud of in your implementation that you might want us to focus on

#### How we'll assess your work:
- Accuracy/RMSE of your model when predicting on held-out data
- How well various edge cases are handled when testing on held-out data. For example, if the held-out data contains:
    - A new column that wasn't present in the dataset given to you
    - New value in a categorical field that wasn't seen in the dataset given to you
    - NA values
- Efficiency of the code. 
    - Is it easy to understand? 
    - Are the variable names descriptive? 
    - Are there any variables created that aren't used? 
    - Is redundant code replaced with function calls? 
    - Is vectorized implementation used instead of nested for loops? 
    - Are classes defined and objects created where applicable? 
    - Are packages used to perform tasks instead of implementing them from scratch?
    
**NOTE:** Your stored models, once loaded, should *just work* when fed with our held-out data (which looks similar to the data we've given you). We won't do any preprocessing before we feed it into the model's `predict(X)` function; `predict(X)` should handle the preprocessing. Pay particular attention to handling the edge cases we've talked about.

Feel free to ask questions to clarify things. Submit everything you tried, not just the things that worked. I encourage you to try and showcase your talents. The more you go above and beyond what's expected, the more impressed we'll be. **Bonus points if you fit Keras/Tensorflow/Pytorch/Caffe models** in addition to your Linear/Tree-based models.

## 0. Import dependencies

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing as scale
from sklearn.utils import resample
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import mean_squared_error, accuracy_score, average_precision_score, precision_score, f1_score,recall_score, roc_auc_score

## Task 1
`predictive_maintenance_dataset.csv` is a file that contains parameters and settings (`operational_setting_1`, `operational_setting_2`, `sensor_measurement_1`, `sensor_measurement_2`, etc.) for many wind turbines. There is a column called `unit_number` which specifies which turbine it is, and one called `status`, in which a value of 1 means the turbine broke down that day, and 0 means it didn't. Your task is to create a model that, when fed with operational settings and sensor measurements (`unit_number` and `time_stamp` will *not* be fed in), outputs 1 if the turbine will break down within the next 40 days, and 0 if not.

**NOTE:** The model should output 1 if the turbine is anywhere between 40 and 0 days away from failure, not *only* 40 days from failure.

In [2]:
## What the data that we'll feed into your model's predict(X) function will look like:
# Notice what the operational_setting_3 column looks like
df_X = pd.read_csv("predictive_maintenance_dataset.csv").drop(labels=['status', 'unit_number', 'time_stamp'], axis='columns')
df_X

,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,42.0007,0.8415,High,445.00,NaN,1362.47,1143.17,3.91,5.70,142.53,...,133.75,2388.50,8129.92,9.1182,NaN,332.0,2212.0,100.00,10.77,6.5717
1,-0.0023,0.0004,High,518.67,642.33,1581.03,1400.06,14.62,21.61,554.60,...,522.19,2388.00,8135.70,8.3817,0.03,393.0,2388.0,100.00,39.07,23.3958
2,NaN,0.6216,Low,462.54,536.71,1250.87,1037.52,7.05,9.00,174.56,...,163.11,2028.06,7867.90,10.8827,NaN,306.0,1915.0,84.93,14.33,8.6202
3,42.0006,NaN,High,NaN,549.28,1349.42,1114.02,3.91,5.71,137.97,...,130.58,2387.71,8074.81,9.3776,0.02,NaN,2212.0,100.00,10.60,6.2614
4,-0.0016,0.0004,High,518.67,643.84,1604.53,1431.41,14.62,21.61,551.30,...,519.44,2388.24,8135.95,8.5223,0.03,396.0,2388.0,100.00,38.39,23.0682
5,25.0046,0.6219,Low,462.54,536.72,NaN,1047.79,7.05,9.03,175.36,...,164.97,2028.40,7880.19,10.8625,0.02,308.0,1915.0,84.93,14.38,8.6381
6,NaN,0.6200,Low,462.54,536.79,1267.31,1045.78,7.05,9.03,174.81,...,165.05,2028.37,7881.95,10.9150,0.02,307.0,1915.0,84.93,14.18,8.5752
7,42.0053,0.8400,High,445.00,548.84,1348.71,1119.73,3.91,5.71,138.95,...,130.38,2387.86,8079.78,9.3526,0.02,329.0,2212.0,100.00,10.64,6.5382
8,0.0029,-0.0003,High,NaN,642.48,1588.88,1393.88,14.62,21.61,NaN,...,522.01,2388.06,NaN,8.3743,0.03,392.0,2388.0,100.00,38.95,23.4351
9,10.0008,0.2504,High,489.05,604.49,1498.95,1309.51,10.52,15.49,394.85,...,371.56,2388.09,8128.11,NaN,0.03,368.0,2319.0,100.00,28.48,17.2737


### 1. Import data

In [215]:
df = pd.read_csv("predictive_maintenance_dataset.csv").sort_values(by = ['unit_number', 'time_stamp'], ascending = True).reset_index()
df

,index,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,73382,2,2017-04-01 12:00:00,0,-0.0018,0.0006,High,518.67,641.89,1583.84,...,522.33,2388.06,8137.72,8.3905,0.03,391.0,2388.0,100.00,38.94,23.4585
1,90923,2,2017-04-02 12:00:00,0,0.0043,-0.0003,High,518.67,641.82,1587.05,...,522.70,2387.98,8131.09,8.4167,0.03,NaN,2388.0,100.00,39.06,23.4085
2,82527,2,2017-04-03 12:00:00,0,0.0018,0.0003,High,518.67,641.55,1588.32,...,522.58,2387.99,8140.58,8.3802,0.03,391.0,2388.0,100.00,39.11,23.4250
3,96521,2,2017-04-04 12:00:00,0,0.0035,-0.0004,High,518.67,641.68,1584.15,...,522.49,2387.93,8140.44,8.4018,0.03,391.0,2388.0,100.00,39.13,23.5027
4,73137,2,2017-04-05 12:00:00,0,0.0005,0.0004,High,518.67,641.73,1579.03,...,522.27,2387.94,8136.67,8.3867,0.03,390.0,2388.0,100.00,39.18,23.4234
5,6093,2,2017-04-06 12:00:00,0,-0.0010,0.0004,High,518.67,641.30,1577.50,...,522.80,2387.99,8133.65,8.3800,0.03,392.0,2388.0,100.00,39.15,23.4270
6,91573,2,2017-04-07 12:00:00,0,0.0001,-0.0002,High,518.67,642.03,1587.49,...,522.14,2388.04,8136.33,8.3941,0.03,391.0,2388.0,100.00,39.10,23.4718
7,77471,2,2017-04-08 12:00:00,0,0.0015,-0.0004,High,518.67,642.55,1590.41,...,522.77,NaN,NaN,8.3861,0.03,391.0,2388.0,100.00,NaN,23.4381
8,93541,2,2017-04-09 12:00:00,0,0.0017,-0.0004,High,518.67,641.98,1581.99,...,522.40,2387.98,8145.29,8.3868,0.03,390.0,2388.0,100.00,39.06,23.4875
9,30788,2,2017-04-10 12:00:00,0,NaN,0.0002,High,518.67,NaN,1586.37,...,521.99,2387.97,8138.64,8.3982,0.03,391.0,2388.0,100.00,NaN,23.6005


In [177]:
df.groupby('unit_number').count().where(df['unit_number']<40) # No unit number values less than 40

,index,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
unit_number,,,,,,,,,,,,,,,,,,,,,
2,287,287,287,275,271,278,275,271,279,274,...,280,279,280,273,269,275,269,276,273,270
3,179,179,179,176,170,176,173,167,175,169,...,166,164,175,168,165,171,168,172,172,172
4,189,189,189,178,180,177,179,179,176,179,...,184,184,178,174,187,175,183,183,181,184
5,269,269,269,254,253,249,256,254,257,258,...,252,255,260,258,256,254,256,260,258,258
6,188,188,188,183,175,178,177,178,174,176,...,179,172,177,180,181,180,178,179,179,175
7,259,259,259,247,252,246,246,246,243,249,...,237,247,250,248,250,253,245,250,243,242
8,150,150,150,142,138,146,142,146,140,138,...,147,138,139,144,144,139,142,146,143,139
9,201,201,201,190,193,190,188,188,191,185,...,189,193,191,186,194,189,189,191,194,191
10,222,222,222,209,211,213,213,212,211,217,...,213,217,213,217,209,211,211,215,215,213


### 2. Explore data

Are there any null columns? 

In [198]:
# Check for columns with Null values
nullcols = []

for col in df.columns:
    nbnull = (df[col].isnull()*1).sum()
    if (nbnull>0): 
        t = type(df[df[col].notnull()][col].iat[0]) # type of first non-null value
        nullcols.append([col,t])
        print(col, nbnull, t)

operational_setting_1 7141 <class 'numpy.float64'>
operational_setting_2 7196 <class 'numpy.float64'>
operational_setting_3 7227 <class 'str'>
sensor_measurement_1 7209 <class 'numpy.float64'>
sensor_measurement_2 7198 <class 'numpy.float64'>
sensor_measurement_3 7190 <class 'numpy.float64'>
sensor_measurement_4 7335 <class 'numpy.float64'>
sensor_measurement_5 7244 <class 'numpy.float64'>
sensor_measurement_6 7444 <class 'numpy.float64'>
sensor_measurement_7 7213 <class 'numpy.float64'>
sensor_measurement_8 7276 <class 'numpy.float64'>
sensor_measurement_9 7207 <class 'numpy.float64'>
sensor_measurement_10 7191 <class 'numpy.float64'>
sensor_measurement_11 7180 <class 'numpy.float64'>
sensor_measurement_12 7227 <class 'numpy.float64'>
sensor_measurement_13 7115 <class 'numpy.float64'>
sensor_measurement_14 7068 <class 'numpy.float64'>
sensor_measurement_15 7257 <class 'numpy.float64'>
sensor_measurement_16 7059 <class 'numpy.float64'>
sensor_measurement_17 7167 <class 'numpy.float64'>

That's a lot of empty values! 

### Categorical value

In [217]:
df = df.dropna()
df

,index,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
0,73382,2,2017-04-01 12:00:00,0,-0.0018,0.0006,High,518.67,641.89,1583.84,...,522.33,2388.06,8137.72,8.3905,0.03,391.0,2388.0,100.00,38.94,23.4585
2,82527,2,2017-04-03 12:00:00,0,0.0018,0.0003,High,518.67,641.55,1588.32,...,522.58,2387.99,8140.58,8.3802,0.03,391.0,2388.0,100.00,39.11,23.4250
3,96521,2,2017-04-04 12:00:00,0,0.0035,-0.0004,High,518.67,641.68,1584.15,...,522.49,2387.93,8140.44,8.4018,0.03,391.0,2388.0,100.00,39.13,23.5027
8,93541,2,2017-04-09 12:00:00,0,0.0017,-0.0004,High,518.67,641.98,1581.99,...,522.40,2387.98,8145.29,8.3868,0.03,390.0,2388.0,100.00,39.06,23.4875
10,34774,2,2017-04-11 12:00:00,0,0.0006,0.0004,High,518.67,642.26,1589.21,...,522.20,2388.03,8135.85,8.4188,0.03,392.0,2388.0,100.00,39.23,23.3888
15,101000,2,2017-04-16 12:00:00,0,-0.0010,0.0003,High,518.67,641.81,1581.99,...,522.49,2387.95,8133.45,8.3805,0.03,392.0,2388.0,100.00,39.01,23.4462
19,54533,2,2017-04-20 12:00:00,0,0.0042,-0.0004,High,518.67,642.34,1582.94,...,522.54,2387.94,8139.83,8.3698,0.03,391.0,2388.0,100.00,39.02,23.3821
20,125449,2,2017-04-21 12:00:00,0,-0.0023,0.0001,High,518.67,642.05,1579.51,...,522.58,2388.01,8136.57,8.3954,0.03,391.0,2388.0,100.00,39.24,23.5014
22,44034,2,2017-04-23 12:00:00,0,-0.0013,0.0003,High,518.67,642.37,1580.89,...,522.26,2388.07,8131.57,8.3937,0.03,393.0,2388.0,100.00,39.10,23.2867
23,38522,2,2017-04-24 12:00:00,0,-0.0019,-0.0000,High,518.67,641.99,1580.97,...,522.25,2387.97,8136.16,8.3866,0.03,391.0,2388.0,100.00,39.18,23.3981


 Consider replacement with mode and creating dummy variables 

In [214]:
df['operational_setting_3'].fillna(df['operational_setting_3'].mode()[0], inplace=True)

Converting to dummy variable to numerically quantify categories
and further reduce variables by only including the high column to indicate if the load is high or low (1 or 0)

In [200]:
df = pd.concat([df.drop('operational_setting_3', axis=1), pd.get_dummies(df.operational_setting_3)], axis=1)

### Numerical values

Some 7000 values are missing out of 144000, that's about 5%, a significant number. This could be valuable information that otherwise may skew our data if not used. 

There are several ways we can approach the missing numerical values. We could use the mean or median values for the entire data set, or narrow down to those values of the individual units. 

Interestingly, the 'status' column gives us a score of how often each turbine breaks down. 

So now we just fill in the missing nan's with the average values of each individual tubine

In [201]:
df.fillna(df.mean(axis=0), axis=0, inplace=True)

,index,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
0,73382,2,2017-04-01 12:00:00,0,-0.001800,0.000600,518.670000,641.890000,1583.840000,1391.2800,...,8137.720000,8.39050,0.030000,391.000000,2388.000000,100.000000,38.940000,23.4585,1,0
1,90923,2,2017-04-02 12:00:00,0,0.004300,-0.000300,518.670000,641.820000,1587.050000,1393.1300,...,8131.090000,8.41670,0.030000,360.999569,2388.000000,100.000000,39.060000,23.4085,1,0
2,82527,2,2017-04-03 12:00:00,0,0.001800,0.000300,518.670000,641.550000,1588.320000,1398.9600,...,8140.580000,8.38020,0.030000,391.000000,2388.000000,100.000000,39.110000,23.4250,1,0
3,96521,2,2017-04-04 12:00:00,0,0.003500,-0.000400,518.670000,641.680000,1584.150000,1396.0800,...,8140.440000,8.40180,0.030000,391.000000,2388.000000,100.000000,39.130000,23.5027,1,0
4,73137,2,2017-04-05 12:00:00,0,0.000500,0.000400,518.670000,641.730000,1579.030000,1402.5200,...,8136.670000,8.38670,0.030000,390.000000,2388.000000,100.000000,39.180000,23.4234,1,0
5,6093,2,2017-04-06 12:00:00,0,-0.001000,0.000400,518.670000,641.300000,1577.500000,1396.7600,...,8133.650000,8.38000,0.030000,392.000000,2388.000000,100.000000,39.150000,23.4270,1,0
6,91573,2,2017-04-07 12:00:00,0,0.000100,-0.000200,518.670000,642.030000,1587.490000,1400.6500,...,8136.330000,8.39410,0.030000,391.000000,2388.000000,100.000000,39.100000,23.4718,1,0
7,77471,2,2017-04-08 12:00:00,0,0.001500,-0.000400,518.670000,642.550000,1590.410000,1262.2954,...,8089.449809,8.38610,0.030000,391.000000,2388.000000,100.000000,26.050472,23.4381,1,0
8,93541,2,2017-04-09 12:00:00,0,0.001700,-0.000400,518.670000,641.980000,1581.990000,1395.0100,...,8145.290000,8.38680,0.030000,390.000000,2388.000000,100.000000,39.060000,23.4875,1,0
9,30788,2,2017-04-10 12:00:00,0,17.075966,0.000200,518.670000,597.771761,1586.370000,1394.8600,...,8138.640000,8.39820,0.030000,391.000000,2388.000000,100.000000,26.050472,23.6005,1,0


### Setting Labels

We have an interesting case here: where we're checking if a turbine is going to fail in 40 days or less. So essentially we're trying to figure out a problem where given all the parameters what is the likelihood that a certain unit fails within a 40 day timespan. 


So we just have to identify the date the turbines failed and mark any other data point going back up to a maximum of 40 days as a failure as well.

In [218]:
df.groupby(['status']).count()

,index,unit_number,time_stamp,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
status,,,,,,,,,,,,,,,,,,,,,
0,41880,41880,41880,41880,41880,41880,41880,41880,41880,41880,...,41880,41880,41880,41880,41880,41880,41880,41880,41880,41880
1,196,196,196,196,196,196,196,196,196,196,...,196,196,196,196,196,196,196,196,196,196


In [184]:
#df = pd.read_csv("processed_dataset.csv")

In [223]:
df

,index,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,...,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,status_40
0,73382,2,2017-04-01 12:00:00,0,-0.0018,0.0006,High,518.67,641.89,1583.84,...,2388.06,8137.72,8.3905,0.03,391.0,2388.0,100.00,38.94,23.4585,0
2,82527,2,2017-04-03 12:00:00,0,0.0018,0.0003,High,518.67,641.55,1588.32,...,2387.99,8140.58,8.3802,0.03,391.0,2388.0,100.00,39.11,23.4250,0
3,96521,2,2017-04-04 12:00:00,0,0.0035,-0.0004,High,518.67,641.68,1584.15,...,2387.93,8140.44,8.4018,0.03,391.0,2388.0,100.00,39.13,23.5027,0
8,93541,2,2017-04-09 12:00:00,0,0.0017,-0.0004,High,518.67,641.98,1581.99,...,2387.98,8145.29,8.3868,0.03,390.0,2388.0,100.00,39.06,23.4875,0
10,34774,2,2017-04-11 12:00:00,0,0.0006,0.0004,High,518.67,642.26,1589.21,...,2388.03,8135.85,8.4188,0.03,392.0,2388.0,100.00,39.23,23.3888,0
15,101000,2,2017-04-16 12:00:00,0,-0.0010,0.0003,High,518.67,641.81,1581.99,...,2387.95,8133.45,8.3805,0.03,392.0,2388.0,100.00,39.01,23.4462,0
19,54533,2,2017-04-20 12:00:00,0,0.0042,-0.0004,High,518.67,642.34,1582.94,...,2387.94,8139.83,8.3698,0.03,391.0,2388.0,100.00,39.02,23.3821,0
20,125449,2,2017-04-21 12:00:00,0,-0.0023,0.0001,High,518.67,642.05,1579.51,...,2388.01,8136.57,8.3954,0.03,391.0,2388.0,100.00,39.24,23.5014,0
22,44034,2,2017-04-23 12:00:00,0,-0.0013,0.0003,High,518.67,642.37,1580.89,...,2388.07,8131.57,8.3937,0.03,393.0,2388.0,100.00,39.10,23.2867,0
23,38522,2,2017-04-24 12:00:00,0,-0.0019,-0.0000,High,518.67,641.99,1580.97,...,2387.97,8136.16,8.3866,0.03,391.0,2388.0,100.00,39.18,23.3981,0


In [219]:
df['status_40'] = df['status'].replace(0, np.NaN) #Let's replace all the 0s with NaNs and then we work backwords

In [220]:
df['status_40'] = df['status'].fillna(method='bfill', limit=40) # fill backward up to 40days. Thankfully the data is frequent and daily
df['status_40'] = df['status'].fillna(int(0)) #fill the rest with zeros

In [225]:
df['status_40'].sum()

196

In [222]:
df.groupby(['status_40']).count()

,index,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
status_40,,,,,,,,,,,,,,,,,,,,,
0,41880,41880,41880,41880,41880,41880,41880,41880,41880,41880,...,41880,41880,41880,41880,41880,41880,41880,41880,41880,41880
1,196,196,196,196,196,196,196,196,196,196,...,196,196,196,196,196,196,196,196,196,196


In [204]:
df = df.drop(['time_stamp', 'unit_number'], axis = 1)
status = df['status']
df = df.drop(['status'], axis = 1)

In [103]:
df = df.drop(['time_stamp', 'unit_number'], axis = 1)

In [13]:
df_work = df #saving the filled dataframe in a separate variable to work

In [14]:
df 

,status,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
73382,0.0,-0.001800,0.000600,518.670000,641.890000,1583.840000,1391.2800,14.620000,21.600000,554.530000,...,8137.720000,8.39050,0.030000,391.000000,2388.000000,100.000000,38.940000,23.4585,1,0
90923,0.0,0.004300,-0.000300,518.670000,641.820000,1587.050000,1393.1300,14.620000,21.610000,554.770000,...,8131.090000,8.41670,0.030000,360.999569,2388.000000,100.000000,39.060000,23.4085,1,0
82527,0.0,0.001800,0.000300,518.670000,641.550000,1588.320000,1398.9600,14.620000,21.600000,555.140000,...,8140.580000,8.38020,0.030000,391.000000,2388.000000,100.000000,39.110000,23.4250,1,0
96521,0.0,0.003500,-0.000400,518.670000,641.680000,1584.150000,1396.0800,14.620000,21.610000,554.250000,...,8140.440000,8.40180,0.030000,391.000000,2388.000000,100.000000,39.130000,23.5027,1,0
73137,0.0,0.000500,0.000400,518.670000,641.730000,1579.030000,1402.5200,14.620000,14.487827,555.120000,...,8136.670000,8.38670,0.030000,390.000000,2388.000000,100.000000,39.180000,23.4234,1,0
6093,0.0,-0.001000,0.000400,518.670000,641.300000,1577.500000,1396.7600,14.620000,21.610000,554.980000,...,8133.650000,8.38000,0.030000,392.000000,2388.000000,100.000000,39.150000,23.4270,1,0
91573,0.0,0.000100,-0.000200,518.670000,642.030000,1587.490000,1400.6500,14.620000,21.610000,554.840000,...,8136.330000,8.39410,0.030000,391.000000,2388.000000,100.000000,39.100000,23.4718,1,0
77471,0.0,0.001500,-0.000400,518.670000,642.550000,1590.410000,1262.2954,14.620000,21.610000,554.650000,...,8089.449809,8.38610,0.030000,391.000000,2388.000000,100.000000,26.050472,23.4381,1,0
93541,0.0,0.001700,-0.000400,518.670000,641.980000,1581.990000,1395.0100,14.620000,21.600000,554.690000,...,8145.290000,8.38680,0.030000,390.000000,2388.000000,100.000000,39.060000,23.4875,1,0
30788,0.0,17.075966,0.000200,518.670000,597.771761,1586.370000,1394.8600,14.620000,21.600000,554.630000,...,8138.640000,8.39820,0.030000,391.000000,2388.000000,100.000000,26.050472,23.6005,1,0


In [15]:
columns_to_normalize = df.columns[1:24] 



standard_sc = scale.StandardScaler()
x_std = standard_sc.fit_transform(df)
df_scaled = pd.DataFrame(x_std)

In [16]:
df_scaled.describe()

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
count,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,...,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05,1.442030e+05
mean,-4.815160e-14,-2.407497e-15,2.686961e-14,1.197724e-13,5.664696e-15,1.888140e-15,-4.368540e-15,-1.840133e-15,8.059762e-14,1.098348e-15,...,-6.605802e-15,-1.318948e-15,-1.287545e-14,-1.627356e-14,2.725349e-15,6.726376e-14,-1.380750e-15,6.966404e-16,-4.265057e-14,4.265057e-14
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-4.684823e-01,-1.059865e+00,-1.133609e+00,-1.386581e+00,-1.503193e+00,-1.954478e+00,-1.795655e+00,-1.448505e+00,-1.404439e+00,-1.324699e+00,...,-3.070024e+00,-1.220100e+00,-1.072788e+00,-1.951242e+00,-2.590864e+00,-2.978932e+00,-1.392082e+00,-1.393568e+00,-2.985677e+00,-3.349324e-01
25%,-4.684823e-01,-1.059239e+00,-1.131381e+00,-1.236909e+00,-1.148943e+00,-9.492222e-01,-1.010243e+00,-1.071113e+00,-1.033334e+00,-1.090434e+00,...,-2.183847e-01,-8.299699e-01,-1.072788e+00,-9.590779e-01,-4.510524e-01,3.533196e-01,-1.021514e+00,-1.022292e+00,3.349324e-01,-3.349324e-01
50%,-4.684823e-01,1.388497e-14,-4.858593e-13,9.834253e-02,1.790132e-01,1.805076e-01,3.252236e-14,5.926707e-13,8.985701e-13,8.364575e-15,...,2.641044e-01,-1.455905e-14,3.527769e-12,1.653750e-01,3.198561e-01,3.533196e-01,-1.774039e-14,-4.930500e-15,3.349324e-01,-3.349324e-01
75%,-4.684823e-01,1.111943e+00,1.207995e+00,1.096831e+00,1.074774e+00,1.024485e+00,1.047802e+00,1.125938e+00,1.134367e+00,1.130136e+00,...,6.300779e-01,3.935588e-01,9.801296e-01,1.025251e+00,8.169840e-01,3.533196e-01,1.117796e+00,1.118489e+00,3.349324e-01,-3.349324e-01
max,2.134552e+00,1.546685e+00,1.213567e+00,1.096831e+00,1.142343e+00,1.291315e+00,1.346519e+00,1.125938e+00,1.134367e+00,1.233613e+00,...,2.601165e+00,2.755787e+00,9.801296e-01,1.256756e+00,8.169840e-01,3.533196e-01,1.212410e+00,1.215329e+00,3.349324e-01,2.985677e+00


In [94]:
df['status']

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
144173    0
144174    0
144175    0
144176    0
144177    0
144178    0
144179    0
144180    0
144181    0
144182    0
144183    0
144184    0
144185    0
144186    0
144187    0
144188    0
144189    0
144190    0
144191    0
144192    0
144193    0
144194    0
144195    0
144196    0
144197    0
144198    0
144199    0
144200    0
144201    0
144202    1
Name: status, Length: 144203, dtype: int64

In [164]:
df.to_csv("processed_dataset.csv")

### Modelling

In [205]:
xtrain, xtest, ytrain, ytest = train_test_split(df, status, test_size = 0.2, random_state = 19 )
#xtrain, xval, ytrain, yval = train_test_split(df, status, test_size=0.25)

In [206]:
ytrain=ytrain.astype(int)
ytest=ytest.astype(int)

In [193]:
xtrain.describe()

,Unnamed: 0,index,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
count,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,...,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000,108152.000000
mean,72082.616808,72020.966233,17.056853,0.406311,486.166962,597.810253,1468.294044,1262.427651,9.938378,14.492112,...,8089.414250,9.048918,0.025227,361.032868,2274.797390,98.404316,26.053613,15.638743,0.899364,0.100636
std,41623.952803,41678.329196,16.110502,0.358808,29.641847,41.424112,115.121605,132.739424,4.157474,6.274496,...,78.487472,0.731235,0.004871,30.221218,138.601644,4.519922,11.410735,6.837914,0.300848,0.300848
min,0.000000,0.000000,-0.008600,-0.000600,445.000000,535.480000,1242.980000,1023.770000,3.910000,5.670000,...,7850.960000,8.156300,0.020000,302.000000,1915.000000,84.930000,10.160000,6.100800,0.000000,0.000000
25%,36042.750000,35915.750000,0.001400,0.000200,449.440000,550.170000,1358.790000,1128.130000,5.480000,8.000000,...,8072.147500,8.441700,0.020000,332.000000,2212.000000,100.000000,14.390000,8.635500,1.000000,0.000000
50%,71986.500000,72058.500000,17.075966,0.406348,489.050000,605.210000,1488.960000,1262.295400,9.935961,14.487827,...,8109.800000,9.049490,0.025226,366.000000,2319.000000,100.000000,26.050472,15.631200,1.000000,0.000000
75%,108097.750000,108083.250000,35.000100,0.840000,518.670000,642.310000,1586.150000,1401.470000,14.620000,21.610000,...,8138.870000,9.337100,0.030000,392.000000,2388.000000,100.000000,38.810000,23.287300,1.000000,0.000000
max,144202.000000,144202.000000,42.008000,0.842000,518.670000,645.110000,1615.010000,1441.160000,14.620000,21.610000,...,8293.720000,11.066300,0.030000,399.000000,2388.000000,100.000000,39.890000,23.950500,1.000000,1.000000


In [157]:
xtest

,index,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
112254,47990,41.998300,0.840000,445.000000,550.450000,1357.090000,1140.4300,3.910000,5.710000,137.93,...,8081.700000,9.43030,0.020000,331.000000,2212.000000,100.000000,10.630000,6.2737,1,0
101539,113866,42.002100,0.842000,445.000000,549.180000,1345.360000,1117.4000,3.910000,5.700000,138.39,...,8074.880000,9.39270,0.020000,330.000000,2212.000000,100.000000,10.520000,6.4533,1,0
72074,11120,17.075966,0.000200,518.670000,642.200000,1586.060000,1405.0900,9.935961,21.580000,550.65,...,8089.449809,8.44480,0.030000,391.000000,2388.000000,100.000000,26.050472,23.2921,1,0
79251,118337,-0.001100,0.000100,518.670000,642.730000,1582.280000,1396.5500,14.620000,14.487827,553.61,...,8131.220000,8.36170,0.030000,393.000000,2388.000000,100.000000,39.070000,23.3611,1,0
102502,65767,35.000400,0.840700,449.440000,555.190000,1350.230000,1262.2954,5.480000,7.980000,193.61,...,8059.900000,9.29700,0.020000,332.000000,2274.604843,100.000000,14.820000,8.8429,1,0
55949,52637,0.000000,0.000000,518.670000,643.000000,1597.090000,1423.3900,14.620000,21.610000,552.22,...,8168.400000,8.47570,0.030000,396.000000,2388.000000,100.000000,38.400000,23.1119,1,0
65426,55706,17.075966,0.000000,486.132686,642.900000,1608.010000,1417.0400,14.620000,14.487827,552.83,...,8199.570000,8.48710,0.030000,395.000000,2388.000000,98.402123,38.620000,23.0893,1,0
69266,134913,0.002900,0.000300,518.670000,642.880000,1589.560000,1412.4000,14.620000,21.610000,553.22,...,8154.490000,8.45680,0.030000,393.000000,2388.000000,100.000000,38.710000,23.1353,1,0
115518,25959,17.075966,0.840000,449.440000,555.520000,1372.470000,1135.4400,5.480000,8.000000,194.61,...,8066.540000,9.33280,0.020000,331.000000,2223.000000,100.000000,14.760000,8.9531,1,0
114311,16428,0.000800,0.000000,518.670000,642.520000,1583.290000,1396.9500,14.620000,21.600000,556.80,...,8143.300000,8.37170,0.030000,391.000000,2388.000000,100.000000,39.110000,23.3853,1,0


In [158]:
xtrain

,index,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
122825,104878,10.005600,0.250800,489.050000,604.860000,1498.270000,1304.4500,10.520000,15.490000,361.227447,...,8114.920000,8.67530,0.030000,368.000000,2319.000000,100.000000,28.540000,17.2360,1,0
101939,123544,0.001200,0.000000,518.670000,642.310000,1590.580000,1262.2954,14.620000,21.610000,553.490000,...,8147.180000,8.43300,0.030000,392.000000,2388.000000,100.000000,38.920000,23.3473,1,0
61700,142655,0.002500,0.000000,518.670000,642.460000,1592.110000,1407.7600,14.620000,21.610000,552.980000,...,8129.020000,8.39850,0.030000,394.000000,2388.000000,100.000000,38.860000,23.2945,1,0
94639,1751,42.005200,0.840900,445.000000,550.090000,1355.710000,1135.3900,3.910000,5.720000,139.370000,...,8092.470000,9.35820,0.020000,333.000000,2212.000000,100.000000,10.830000,6.2895,1,0
92413,138111,20.007200,0.700000,491.190000,607.580000,1482.340000,1247.5700,9.350000,13.660000,334.110000,...,8054.240000,9.22390,0.020000,367.000000,2274.604843,100.000000,24.510000,14.6635,1,0
121716,36789,35.005800,0.406348,449.440000,555.560000,1352.670000,1129.4000,5.480000,7.960000,194.530000,...,8069.100000,9.29100,0.020000,334.000000,2274.604843,100.000000,14.820000,8.9744,1,0
72571,115783,-0.002400,-0.000300,518.670000,642.060000,1582.010000,1395.2400,14.620000,21.610000,554.300000,...,8149.770000,8.42110,0.030000,360.999569,2388.000000,100.000000,39.160000,23.4640,1,0
129299,18722,19.998800,0.700000,491.190000,607.990000,1484.250000,1261.0700,9.350000,13.650000,334.780000,...,8057.170000,9.20260,0.020000,366.000000,2324.000000,100.000000,24.510000,14.7610,1,0
2592,48392,0.002700,-0.000300,518.670000,642.980000,1589.210000,1411.2200,14.620000,21.610000,553.740000,...,8143.270000,9.04949,0.030000,393.000000,2388.000000,100.000000,38.640000,23.2067,1,0
143061,1893,19.998800,0.700500,491.190000,607.900000,1494.830000,1251.1600,9.350000,13.660000,334.990000,...,8071.320000,9.21770,0.020000,365.000000,2324.000000,100.000000,24.500000,15.6312,1,0


In [159]:
xtrain.describe()

,index,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
count,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,...,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000,21630.000000
mean,72317.217060,16.965869,0.404589,486.287717,598.113169,1468.903422,1262.846410,9.952902,14.519098,362.357538,...,8089.818357,9.044212,0.025246,361.210426,2275.272196,98.419107,26.112519,15.671210,0.900555,0.099445
std,41684.983917,16.115892,0.358629,29.661980,41.390378,114.778478,132.593244,4.165560,6.278146,169.842955,...,78.289756,0.729826,0.004868,30.142639,138.281931,4.504745,11.404956,6.846028,0.299266,0.299266
min,0.000000,-0.008600,-0.000600,445.000000,535.670000,1243.660000,1025.960000,3.910000,5.680000,136.430000,...,7851.030000,8.156300,0.020000,302.000000,1915.000000,84.930000,10.240000,6.149000,0.000000,0.000000
25%,36186.250000,0.001400,0.000200,449.440000,550.192500,1359.130000,1128.292500,5.480000,8.000000,176.010000,...,8072.602500,8.441400,0.020000,332.000000,2212.000000,100.000000,14.390000,8.635100,1.000000,0.000000
50%,72503.000000,17.075966,0.406348,489.050000,605.240000,1489.390000,1262.295400,9.935961,14.487827,361.227447,...,8112.550000,9.049490,0.025226,366.000000,2319.000000,100.000000,26.050472,15.631200,1.000000,0.000000
75%,108541.250000,34.999700,0.840000,518.670000,642.310000,1586.187500,1401.370000,14.620000,21.610000,553.230000,...,8139.040000,9.336575,0.030000,392.000000,2388.000000,100.000000,38.820000,23.289600,1.000000,0.000000
max,144198.000000,42.008000,0.842000,518.670000,644.450000,1613.000000,1439.980000,14.620000,21.610000,570.810000,...,8293.720000,11.057200,0.030000,399.000000,2388.000000,100.000000,39.850000,23.885400,1.000000,1.000000


In [210]:
ytest

112254    0
101539    0
72074     0
79251     0
102502    0
55949     0
65426     0
69266     0
115518    0
114311    0
54433     0
12291     0
102561    0
124725    0
120018    0
80560     0
94952     0
140825    0
21911     0
94769     0
26426     0
51887     0
130425    0
101215    0
95228     0
57381     0
121552    0
14845     0
135009    0
39745     0
         ..
11449     0
138024    0
142529    0
107197    0
93122     0
128688    0
126321    0
124464    0
142181    0
76325     0
19660     0
29202     0
93188     0
89823     0
47996     0
112006    0
139142    0
35088     0
137909    0
130253    0
13228     0
90597     0
61162     0
86173     0
87465     0
74279     0
116020    0
93090     0
137943    0
51612     0
Name: status, Length: 28841, dtype: int32

In [207]:
def logisticRegression(xtrain,xval, ytrain, yval):
    LR = LogisticRegression()
    model = LR.fit(xtrain.values,ytrain.values)
    pred = model.predict(xtest.values).round(2)
    metrics(pred,ytest)


In [212]:
logisticRegression(xtrain,xval, ytrain, yval)

accuracy score:  0.995457855137
RMSE: 0.00454214486322
Recall score:  0.0
average_precision_score:  nan
Precision Score:  0.0
F1_score:  0.0


C:\Users\Prakash14\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
C:\Users\Prakash14\Anaconda3\lib\site-packages\sklearn\metrics\ranking.py:444: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
C:\Users\Prakash14\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [45]:
randomForestClassifier(xtrain,xtest,ytrain,ytest,n_estimators=25,min_samples_split=25,max_depth=5,random_state=72)

accuracy score:  1.0
RMSE: 0.0
Recall score:  1.0
average_precision_score:  1.0
Precision Score:  1.0
F1_score:  1.0
roc_auc_score:  1.0


In [46]:
supportVectorMachine(xtrain,xtest,ytrain,ytest)

accuracy score:  1.0
RMSE: 0.0
Recall score:  1.0
average_precision_score:  1.0
Precision Score:  1.0
F1_score:  1.0
roc_auc_score:  1.0


In [49]:
gaussianNaiveBayes(xtrain,xtest,ytrain,ytest)

In [21]:
def decisionTreeClassifier(xtrain, xtest, ytrain, ytest, random_state):
    DT = decisionTreeClassifier(xtrain, xtest, ytrain, ytest, random_state=12)
    model = DT.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    metrics(ytest,pred)
    

In [22]:
def randomForestClassifier(xtrain,xtest,ytrain,ytest,n_estimators=25,min_samples_split=25,max_depth=5,random_state=72):
    RF = RandomForestClassifier(n_estimators = 25, min_samples_split=25, max_depth =5, random_state=72)
    model = RF.fit(xtrain,ytrain)
    pred = RF.predict(xtest)
    metrics(ytest, pred)

In [23]:
def gaussianNaiveBayes(xtrain,xtest,ytrain,ytest):
    GNB = GaussianNB()
    model = GNB.fit(xtrain,ytrain)
    pred = GNB.predict(xtest)
    metrics = (ytest, pred)

In [24]:
def supportVectorMachine(xtrain,xtest,ytrain,ytest):
    svc = SVC(kernel='linear')
    model = svc.fit(xtrain,ytrain)
    pred = svc.predict(xtest)
    metrics(ytest,pred)

In [211]:
def metrics(ytest, pred):
    """
    Function to evaluate models against models 
    """
    print('accuracy score: ', accuracy_score(ytest, pred))
    print('RMSE:', mean_squared_error(ytest,pred))
    print('Recall score: ', recall_score(ytest,pred))
    
    print('average_precision_score: ', average_precision_score(ytest,pred))
    print('Precision Score: ',precision_score(ytest,pred))
    print('F1_score: ',f1_score(ytest, pred))
    #print('roc_auc_score: ', roc_auc_score(ytest, pred))

### Conclusion

100% on everything? That's very fishy! Perhaps, my way of marking due for repair might be doing something to the dataset?
or the extent of data scaling might be? 

## Task 2
`forecasting_dataset.csv` is a file that contains pollution data for a city. Your task is to create a model that, when fed with columns `co_gt`, `nhmc`, `c6h6`, `s2`, `nox`, `s3`, `no2`, `s4`, `s5`, `t`, `rh`, `ah`, and `level`, predicts the value of `y` six hours later.

**NOTE:** In the data we've given you, the value of `y` for a given row is the value of `y` *for the timestamp of that same row*. We're asking you to predict the value of `y` 6 hours *after the timestamp of that row*.

In [100]:
## What the data that we'll feed into your model's predict(X) function will look like:
# Notice what the level column looks like
pd.read_csv("forecasting_dataset.csv").head().drop(labels=['date', 'time', 'y'], axis='columns')

,co_gt,nhmc,c6h6,s2,nox,s3,no2,s4,s5,t,rh,ah,level
0,-200.0,-200.0,7.2,867.0,-200.0,834.0,-200.0,1314.0,891.0,14.8,57.3,0.9603,NaN
1,0.5,-200.0,3.9,704.0,-200.0,861.0,-200.0,1603.0,860.0,24.4,65.0,1.9612,Low
2,3.7,-200.0,23.3,1386.0,NaN,626.0,109.0,2138.0,NaN,23.3,38.6,1.0919,High
3,2.1,-200.0,12.1,1052.0,183.0,779.0,NaN,1690.0,952.0,28.5,27.3,1.0479,High
4,4.4,-200.0,21.7,1342.0,786.0,499.0,206.0,1546.0,2006.0,12.9,54.1,0.8003,High


In [82]:
pd.read_csv("forecasting_dataset.csv").sort_values(by=['date'], ascending = False)

,date,time,y,co_gt,nhmc,c6h6,s2,nox,s3,no2,s4,s5,t,rh,ah,level
1820,9/9/2004,3:00:00,814,NaN,-200.0,2.3,606.0,-200.0,1370.0,-200.0,1131.0,NaN,23.0,33.0,0.9184,Low
3802,9/9/2004,19:00:00,1329,4.3,-200.0,22.3,1360.0,435.0,NaN,175.0,1768.0,1894.0,31.2,25.5,1.1379,High
1545,9/9/2004,10:00:00,1277,3.1,-200.0,19.9,1292.0,590.0,800.0,171.0,1626.0,1481.0,31.0,21.9,0.9708,High
5954,9/9/2004,8:00:00,1289,3.2,NaN,21.4,1335.0,542.0,724.0,130.0,1681.0,1521.0,21.0,35.6,NaN,High
2172,9/9/2004,18:00:00,1425,5.4,-200.0,32.3,1605.0,631.0,660.0,220.0,1920.0,2321.0,35.2,16.6,0.9242,High
7545,9/9/2004,14:00:00,1160,2.5,-200.0,16.2,NaN,NaN,873.0,168.0,1522.0,1162.0,35.1,17.5,0.9733,High
5836,9/9/2004,22:00:00,1089,1.5,-200.0,11.3,1024.0,180.0,831.0,111.0,1452.0,NaN,27.4,31.5,1.1330,High
2564,9/9/2004,9:00:00,1355,3.5,-200.0,21.3,1333.0,590.0,741.0,157.0,1691.0,1605.0,25.0,28.9,0.9025,High
1764,9/9/2004,16:00:00,1166,2.9,-200.0,16.2,1186.0,393.0,878.0,175.0,1500.0,1180.0,38.4,13.8,0.9221,High
6216,9/9/2004,15:00:00,1148,2.4,-200.0,15.0,1149.0,NaN,924.0,152.0,1473.0,1047.0,35.4,16.6,NaN,NaN
